# Analyze Connections Design Images
### Prerequisites
- An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
- Python 3.12.9, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
%pip install openai
%pip install python-dotenv
%pip install pymupdf
%pip install pillow

### Variables in .env file:
- AOAI_ENDPOINT=YOUR-AZURE-OPENAI-O1-ENDPOINT
- AOAI_API_KEY=YOUR-AZURE-OPENAI-O1-APIKEY
- AOAI_O1_DEPLOYMENT_NAME=YOUR-AZURE-OPENAI-O1-NAME
- AOAI_API_VERSION=2024-12-01-preview

#### Import packages and define all functions needed

In [1]:
import os
import fitz  # PyMuPDF
from dotenv import load_dotenv
from openai import AzureOpenAI
import base64
from mimetypes import guess_type
from PIL import Image # Pillow

# Get Azure OpenAI Service settings
load_dotenv(override=True)

# o1 model configuration
aoai_endpoint = os.getenv("AOAI_ENDPOINT")
aoai_apikey = os.getenv("AOAI_API_KEY")
aoai_deployment_name = os.getenv("AOAI_DEPLOYMENT_NAME")
api_version = os.getenv("AOAI_API_VERSION")
print(f'aoai_endpoint: {aoai_endpoint}, aoai_deployment_name: {aoai_deployment_name}')
# Create the AOAI client
aoai_client = AzureOpenAI(
    azure_endpoint = aoai_endpoint, 
    api_key= aoai_apikey,
    api_version=api_version
)

# gpt-4o model configuration
aoai_gpt4o_endpoint = os.getenv("AOAI_GPT4O_ENDPOINT")
aoai_gpt4o_apikey = os.getenv("AOAI_GPT4O_API_KEY")
aoai_gpt4o_deployment_name = os.getenv("AOAI_GPT4O_DEPLOYMENT_NAME")
print(f'aoai_gpt4o_endpoint: {aoai_gpt4o_endpoint}, aoai_deployment_name: {aoai_gpt4o_deployment_name}')
# Create the AOAI client
aoai_gpt4o_client = AzureOpenAI(
    azure_endpoint = aoai_gpt4o_endpoint, 
    api_key= aoai_gpt4o_apikey,
    api_version=api_version
)

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

# Function to analyze an image
def analyze_image(client, deployment_name, image_path, system_prompt, user_prompt, temperature=0.0):
    # Prepare the image to analyze
    data_url = local_image_to_data_url(image_path)
    print(f'Analyzing image {image_path}...\n')

    # Initiate the message with the system prompt
    messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": [
                { 
                    "type": "text",
                    "text": user_prompt
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": data_url
                    }
                }
                ]
            }
    ]
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages
    )

    print(response)

    return response.model_dump()['choices'][0]['message']['content']

# PROMPTS

# STEP 1: Extract connection types from connnection legend
SYSTEM_PROMPT_CONNECTION_TYPES = """You are an expert engineer in hydraulic schematic design.
The provided image is the hydraulic schematic design page that includes the legend with the connection types.
Your task is to analyze the legend and describe the connection types based on the colors and shapes of the lines in the hydraulic schematic design page.
Provide the connection type names according to the colors and shapes of the lines.
Your resopnse should be in this JSON format:
"connection_types": [
    {
        "connection_style": "color and shape of the connection line",
        "connection_type": "connection name as listed in the legend inside the hydraulic schematic design page"
    }
]
"""
USER_PROMPT_CONNECTION_TYPES = "Analyze this image with the legend information about the connection types and provide the connection name, shapes and colors:"

# STEP 2: Extract connections from schematic design
SYSTEM_PROMPT_CONNECTIONS = """You are an expert engineer in hydraulic schematic design.
The provided image was created from several pages of a document.
The first part, in vertical orientation, contains the legend page(s) with the component symbols and their names.
The last part of the image, in horizontal orientation, is the hydraulic schematic design page.
Your task is to analyze the hydraulic schematic design and describe the type of connections between the components, taking into account the information in the legend with the following details:
- The components are represented by symbols described in the legend or by standard symbols for hydraulic schematic designs.
- The connection type names are the specified in this json: {connection_types}. The colors and shapes of the lines indicate the type of connection.
- The number associated with each component is identified by its symbol as described in the legend pages.

Take your time to read the legend of the hydraulic schematic design and analyze the connections between the components in the hydraulic schematic design page.
Provide the connections type names according to the colors and shapes of the lines and the component numbers based on the information in the connection types provided.
Provide a detailed description of the connections between the components in the hydraulic schematic design page based on the information in the legend.

Your response should be in this JSON format:
"connections": [
   {{
      "component_name": "complete name from the legend",
      "component_number": component number,
      "connected_to": [ 
			{{"component_number": component number,
			  "connection_style": "color and shape of the line",
              "connection_type": "named as listed in the connection types provided",
			}}
		]
   }}
]
"""
USER_PROMPT_CONNECTIONS = "Analyze this image with the legend information and the hydraulic schematic design:"

aoai_endpoint: https://gbb-ea-aoai-swedencentral-shared-reasoning.openai.azure.com/, aoai_deployment_name: o1
aoai_gpt4o_endpoint: https://openai-eastus-asc.openai.azure.com/, aoai_deployment_name: gpt-4o


## Extract pages from pdf to png and Schitching them together

#### Extract PDF pages in PNG files

In [3]:
# Load a PDF document
def read_pdf(pdf_doc):
    doc = fitz.open(pdf_doc)
    print(f"PDF File {os.path.basename(pdf_doc)} has {len(doc)} pages.")
    return doc

# Extract PDF pages to PNG
def extract_pages_as_png_files(output_dir, pdf_file, doc):
    png_files = []
    for page in doc:
        page_num = page.number+1
        img_path = f"{output_dir}/{os.path.splitext(pdf_file)[0]}_page_{page_num}.png"
        page_pix = page.get_pixmap(dpi=300)
        page_pix.save(img_path)
        print(f"\tPage {page_num} saved as {img_path}")
        png_files.append(img_path)
    return png_files

# Extract pages in PNG files
input_dir = '.'
# Create a directory to store the outputs
work_dir = input_dir + "/png_outputs"
os.makedirs(work_dir, exist_ok=True)

# Process every pdf of LENGED
for pdf_doc in os.listdir(input_dir):
    if pdf_doc.lower().endswith('.pdf'): # and pdf_doc.find('_legend') != -1:
        print(f'Processing file [{input_dir}/{pdf_doc}]')
        doc = read_pdf(input_dir + '/' + pdf_doc)
        output_dir = work_dir + '/' + os.path.basename(pdf_doc)[:os.path.basename(pdf_doc).find('_legend')]
        os.makedirs(output_dir, exist_ok=True)
        extract_pages_as_png_files(output_dir, pdf_doc, doc)

Processing file [./462-Piping-and-Instrumentation-Diagrams.pdf]
PDF File 462-Piping-and-Instrumentation-Diagrams.pdf has 42 pages.
	Page 1 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_1.png
	Page 2 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_2.png
	Page 3 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_3.png
	Page 4 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_4.png
	Page 5 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_5.png
	Page 6 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_6.png
	Page 7 saved as ./png_outputs/462-Piping-and-Instrumentation-Diagrams.pd/462-Piping-and-Instrumentation-Diagrams_page_

#### Stitch legend pages and hydraulic diagram page in a single image

In [4]:
import shutil
# Stitch legends and diagram in a single image

# The PDF of legends and the PNGs of design schemas should be in the current directory
schema_names = ['462-Piping', 'abb', 'ML102530301']

for schema_name in schema_names:
  images=[]
  input_dir = os.path.join('.', schema_name)
  # Copy shema image to the output directory
  #shutil.copy(schema_name + '_schema.png', input_dir)

  # Process every PNG
  png_files = []
  for png in os.listdir(input_dir):
      if png.endswith('.png'):
        png_files.append(input_dir + "/" + png)
  print(png_files)

  # Read images and calculate total size
  images = [Image.open(file) for file in png_files]
  widths, heights = zip(*(i.size for i in images))
  max_width = max(widths)
  total_height = sum(heights)

  # Create a new image with total size
  new_image = Image.new('RGB', (max_width, total_height))

  # Paste each image into the new image
  y_offset = 0
  for im in images:
    new_image.paste(im, (0, y_offset))
    y_offset += im.size[1]

  # Save stitched image
  output_path = os.path.join(input_dir, schema_name + '_stitchted.png')
  new_image.save(output_path, 'PNG')
  print(f'Saved {output_path}')

['.\\462-Piping/462-Piping_legend_1.png', '.\\462-Piping/462-Piping_legend_2.png', '.\\462-Piping/462-Piping_schema.png']
Saved .\462-Piping\462-Piping_stitchted.png
['.\\abb/abb_legend_1.png', '.\\abb/abb_schema.png']
Saved .\abb\abb_stitchted.png
['.\\ML102530301/ML102530301_legend_1.png', '.\\ML102530301/ML102530301_legend_2.png', '.\\ML102530301/ML102530301_legend_3.png', '.\\ML102530301/ML102530301_legend_4.png', '.\\ML102530301/ML102530301_legend_5.png', '.\\ML102530301/ML102530301_legend_6.png', '.\\ML102530301/ML102530301_legend_7.png', '.\\ML102530301/ML102530301_legend_8.png', '.\\ML102530301/ML102530301_schema.png']
Saved .\ML102530301\ML102530301_stitchted.png


### STEP 1: Identify the connection types using the legend of connection types

In [ ]:
# IMAGE TO ANALYZE TO EXTRACT CONNECTION TYPES
image_path = "Hydraulic_design/Hydraulic_design_schema.png"

try:
    #connection_types = analyze_image(aoai_gpt4o_client, aoai_gpt4o_deployment_name,
    connection_types = analyze_image(aoai_client, aoai_deployment_name,
                                     image_path,
                                     system_prompt = SYSTEM_PROMPT_CONNECTION_TYPES, 
                                     user_prompt = USER_PROMPT_CONNECTION_TYPES)
    print(f"STEP 1: CONNECTION TYPES:\n {connection_types}")
except Exception as ex:
    print(ex)

Analyzing image Product1/Product1_schema.png...

ChatCompletion(id='chatcmpl-BG8Edw5zkTLIO0k07srnwt1hnYIx0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "connection_types": [\n    {\n      "connection_style": "Gray (solid line)",\n      "connection_type": "PFC Piston Pump Supply (High Pressure)"\n    },\n    {\n      "connection_style": "Red (solid line)",\n      "connection_type": "PFC Piston Pump Supply (Regulated)"\n    },\n    {\n      "connection_style": "Orange (solid line)",\n      "connection_type": "Charge Pump Flow"\n    },\n    {\n      "connection_style": "Blue (solid line)",\n      "connection_type": "Return to Tank / Sump"\n    },\n    {\n      "connection_style": "Green (solid line)",\n      "connection_type": "Twin Flow PFC Piston Pump Supply"\n    },\n    {\n      "connection_style": "Purple (solid line)",\n      "connection_type": "Axle Lube Supply"\n    },\n    {\n      "connection_style": "Brown (solid li

### STEP 2: Identify connections between devices using the symbols from the legend and the connection types using the types identified in step 1

In [ ]:
# IMAGE TO ANALYZE WITH STITCHED IMAGES
image_path = "Hydraulic_design/Hydraulic_design_stitchted.png"

try:
    system_prompt = SYSTEM_PROMPT_CONNECTIONS.format(connection_types=connection_types)
    response = analyze_image(aoai_client,
                             aoai_deployment_name,
                             image_path,
                             system_prompt = system_prompt, 
                             user_prompt = USER_PROMPT_CONNECTIONS)
    print(f"\nSTEP 2: CONNECTIONS:\n{response}")
    
except Exception as ex:
    print(f'ERROR: {ex}')

Analyzing image Product1/Product1_stitchted.png...

ChatCompletion(id='chatcmpl-BG8HUFODoY3uFMVK1YtXBHDp739XC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "connections": [\n    {\n      "component_name": "Main PFC Piston Pump",\n      "component_number": 1,\n      "connected_to": [\n        {\n          "component_number": 2,\n          "connection_style": "Orange (solid line)",\n          "connection_type": "Charge Pump Flow"\n        },\n        {\n          "component_number": 3,\n          "connection_style": "Red (solid line)",\n          "connection_type": "PFC Piston Pump Supply (Regulated)"\n        },\n        {\n          "component_number": 4,\n          "connection_style": "Blue (solid line)",\n          "connection_type": "Return to Tank / Sump"\n        }\n      ]\n    },\n    {\n      "component_name": "Charge Pump",\n      "component_number": 2,\n      "connected_to": [\n        {\n          "component_numbe